# Prohibited punishment violations

Queries [violation](https://data.texas.gov/Social-Services/DFPS-CCL-Non-Compliance-Data/tqgd-mf4x), [operation](https://data.texas.gov/Social-Services/DFPS-CCL-Daycare-and-Residential-Operations-Data/bc5r-88dy) and [assessment](https://data.texas.gov/Social-Services/DFPS-CCL-Inspection-Investigation-Assessment-Data/m5q4-3y3d) datasets downloaded from the state data portal on Feb. 1, 2018, against a spreadsheet of fines provided by the Texas Health and Human Services Commission.

In [1]:
import pandas as pd

In [2]:
non_compliance = pd.read_csv('../src/dfps-2018-02-01/non_compliance.csv').drop('Unnamed: 0', axis=1)
operations = pd.read_csv('../src/dfps-2018-02-01/operations.csv').drop('Unnamed: 0', axis=1)
inspections = pd.read_csv('../src/dfps-2018-02-01/assessment.csv').drop('Unnamed: 0', axis=1)

In [3]:
punishment = non_compliance.assign(
    cruel = lambda x: x.STANDARD_NUMBER_DESCRIPTION.str.lower().isin(
        pd.Series(list(filter(
            lambda x: 'harsh, cruel' in x,
            [a for a in non_compliance.STANDARD_NUMBER_DESCRIPTION.str.lower().unique()]
        )))
    ) | x.STANDARD_NUMBER_DESCRIPTION.isin([
        '746.2805(4) - Prohibited Punishments - Hitting with Hand or Instrument',
        '746.2805(3) - Prohibited Punishments - Pinching, Shaking, or Biting',
        '746.2805(8) - Prohibited Punishments -Placing Child in Locked, Dark Room'
    ])
).merge(
    operations,
    on='OPERATION_ID'
).query(
    'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
).query('cruel == True').append(
    non_compliance.assign(
        cruel = lambda x: x.STANDARD_NUMBER_DESCRIPTION.str.lower().isin(
            pd.Series(list(filter(
                lambda x: 'neglected' in x,
                [a for a in non_compliance.STANDARD_NUMBER_DESCRIPTION.str.lower().unique()]
            )))
        )
    ).merge(
        operations,
        on='OPERATION_ID'
    ).query(
        'OPERATION_TYPE != "Child Placing Agency" & OPERATION_TYPE != "General Residential Operation"'
    ).query('cruel == True').set_index(
        'ACTIVITY_ID'
    )
)[[
    'OPERATION_ID', 'OPERATION_NAME', 'LOCATION_ADDRESS', 'COUNTY',
    'STANDARD_NUMBER_DESCRIPTION',
    'CORRECTED_DATE',
    'NARRATIVE', 'TECHNICAL_ASSISTANCE_GIVEN', 'OPERATION_TYPE',
]].assign(
    in_timeframe = lambda x: pd.to_datetime(x.CORRECTED_DATE).apply(
        lambda x: x >= pd.to_datetime('2016-02-01')
    )
).query('in_timeframe').drop(
    'in_timeframe', axis=1
)

In [4]:
# Export for database
punishment.to_csv(
    '../database/punishment.csv',
    index=False
)

In [5]:
fines = pd.read_excel(
    '../Fines.xlsx',
    skiprows=3,
    skip_footer=1,
    converters={'Operation Number': int}
).rename(
    columns = lambda x: x.replace(' ', '')
)

In [6]:
fines.head()

,YearPenaltyPaid1,OperationNumber,OperationName,AmountFined,AmountPaid
0,2003.0,558023,Levelland Christian School,60.0,60.0
1,2015.0,970046,Southbelt Montessori Learning Center,100.0,100.0
2,2015.0,1544977,United Kidz Academy San Antonio,150.0,150.0
3,2015.0,514294,Lucy's Day Care,50.0,50.0
4,2015.0,531697,Tiny Angels Day Care,50.0,50.0


In [7]:
# Search operations cited for prohibited punishments in fines spreadsheet

fines.assign(
    name_number = lambda x: x.OperationName.astype(str) + x.OperationNumber.astype(str)
).merge(
    punishment.assign(
        name_number = lambda x: x.OPERATION_NAME.astype(str) + x.OPERATION_ID.astype(str)
    ),
    on='name_number'
)

,YearPenaltyPaid1,OperationNumber,OperationName,AmountFined,AmountPaid,name_number,OPERATION_ID,OPERATION_NAME,LOCATION_ADDRESS,COUNTY,STANDARD_NUMBER_DESCRIPTION,CORRECTED_DATE,NARRATIVE,TECHNICAL_ASSISTANCE_GIVEN,OPERATION_TYPE
